<a href="https://colab.research.google.com/github/jjramcar/Data-Science-Projects/blob/main/SpamPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spam Prediction

Here we will explore text message data and create models to predict if a message is spam or not.

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv(r'/content/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [4]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Task 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [2]:
def task_one():
    
    spam_data.dropna(inplace=True) 
    
    spam = spam_data[spam_data['target'] != 0]
    
    return len(spam)/len(spam_data)*100

In [3]:
task_one()

13.406317300789663

### Task 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def task_two(): 
    
    vect = CountVectorizer().fit(X_train)
    
    feat_names = vect.get_feature_names()
    
    lt = [t for t in feat_names if len(t) > 29 ]
    
    lt = lt[0]
      
    return lt

In [7]:
task_two()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'com1win150ppmx3age16subscription'

### Task 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def task_three():
    
    vect = CountVectorizer().fit(X_train)
    
    X_train_vectorized = vect.transform(X_train)
    
    # Train the model
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [9]:
task_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

def task_four():
    
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    feature_names = np.array(vect.get_feature_names())
    
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort() 
    
    smallest = feature_names[sorted_tfidf_index[:20]]
    largest = feature_names[sorted_tfidf_index[:-21:-1]]
    
    small_index = []
    large_index = []
    
    for s, l in zip(smallest, largest):
        small_index.append(s)
        large_index.append(l)
    
    small_data = []
    large_data = []
    
    for s, l in zip(sorted_tfidf_index[:20],sorted_tfidf_index[:-21:-1]):
        small_data.append(X_train_vectorized.sum(axis=0).item(s)) 
        large_data.append(X_train_vectorized.sum(axis=0).item(l))
    
    
    
    #for col, term in enumerate(feature_names):
    #    small_data.append((term,sums[0,col]))
    
    #names = feature_names[sorted_tfidf_index.nonzero()]
        
    
    #for s, l in zip(sorted_tfidf_index[:20], sorted_tfidf_index[:-21:-1]):
    #    for i in range(4179):
    #        if X_train_vectorized[i,s] != 0:
    #            small_data.append(X_train_vectorized[i,s])
    #        if X_train_vectorized[i,l] != 0:
    #            large_data.append(X_train_vectorized[i,l])
        
    #tfidf_matrix= X_train_vectorized.todense().argsort()
    #feature_index = tfidf_matrix[0,:].nonzero()[1]
    #small_data = [tfidf_matrix[0, x] for x in sorted_tfidf_index[:20]]
    #large_data = [tfidf_matrix[0, x] for x in sorted_tfidf_index[:-21:-1]]
    #return dict(tfidf_scores)
    
    small_index[0:17] = sorted(small_index[0:17])
    small_ser = pd.Series(data=small_data, index = small_index)
    large_ser = pd.Series(data=large_data, index = large_index)
           
    #small_ser = sorted(small_ser.items(), key=lambda item: item[1])
    large_ser = sorted(large_ser.items(), key=lambda item: item[1], reverse=True)
    
    large_index1 = []
    large_data1 = []
    
    for i in range(20):
        large_index1.append(large_ser[i][0])
        large_data1.append(large_ser[i][1])
        
    large_index1[15:18] = sorted(large_index1[15:18])
    #small_ser = pd.Series(data= small_ser[:,1], index = small_ser[:,0])
    large_ser1 = pd.Series(data=large_data1, index = large_index1)
    #tfidf_matrix= X_train_vectorized.todense()
    #feature_index = tfidf_matrix[0,:].nonzero()[1]
    #tfidf_scores = zip([feature_names[i] for i in feature_index[:20]], [tfidf_matrix[0, x] for x in feature_index[:20]])  
    
        
   
    #print( X_train_vectorized)
    
    #dict(tfidf_scores)
    
    #return sorted_tfidf_index[:-21:-1]
    
    return (small_ser, large_ser1)

In [12]:
task_four()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, ok           70.873379
 home         34.549724
 where        28.778679
 too          18.491193
 anything     17.038126
 yup          14.361014
 thanx        11.583239
 okie         10.886852
 done         10.872728
 havent        7.242888
 lei           7.127380
 nite          6.183283
 thank         5.334519
 anytime       4.490321
 er            2.511243
 146tf150p     1.000000
 645           1.000000
 beerage       1.000000
 tick          0.980166
 blank  

### Task 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [13]:
def task_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)

   
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
   

    predictions = model.predict(vect.transform(X_test))

    return roc_auc_score(y_test, predictions)

In [14]:
task_five()

0.9416243654822335

### Task 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [15]:
def task_six():
    
    spam = spam_data[spam_data['target'] != 0]
    not_spam = spam_data[spam_data['target'] == 0]
    
    num_of_char_not = 0
    num_of_char_spam = 0
    
    for i in range(len(spam_data)):
        if spam_data['target'][i] == 0:
            num_of_char_not += len(spam_data['text'][i])
        elif spam_data['target'][i] == 1:
            num_of_char_spam += len(spam_data['text'][i])

    
    return (num_of_char_not/len(not_spam), num_of_char_spam/len(spam))

In [16]:
task_six()

(71.02362694300518, 138.8661311914324)

In [17]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Task 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [18]:
from sklearn.svm import SVC

def task_seven():
    vectorizer = TfidfVectorizer(min_df=5)

    X_train_transformed = vectorizer.fit_transform(X_train)
    X_train_transformed_with_length = add_feature(X_train_transformed, X_train.str.len())

    X_test_transformed = vectorizer.transform(X_test)
    X_test_transformed_with_length = add_feature(X_test_transformed, X_test.str.len())

    clf = SVC(C=10000)

    clf.fit(X_train_transformed_with_length, y_train)

    y_predicted = clf.predict(X_test_transformed_with_length)
    
    return roc_auc_score(y_test, y_predicted)

In [19]:
task_seven()

0.9661689557407943

### Task 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [20]:
def task_eight():
    
    spam = spam_data[spam_data['target'] != 0]
    not_spam = spam_data[spam_data['target'] == 0]
    
    digit_spam = 0
    digit_not= 0
    
    for i in range(len(spam_data)):
        if spam_data['target'][i] == 0:
            for s in spam_data['text'][i]:
                if s.isdigit():
                    digit_not = digit_not + 1
        #elif spam_data['target'][i] == 1:
        #    num_of_char_spam += len(spam_data['text'][i])
    
    for i in range(len(spam_data)):
        if spam_data['target'][i] == 1:
            for s in spam_data['text'][i]:
                if s.isdigit():
                    digit_spam = digit_spam + 1
  
            
    
    
    return (digit_not/len(not_spam), digit_spam/len(spam))

In [21]:
task_eight()

(0.2992746113989637, 15.759036144578314)

### Task 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [22]:
from sklearn.linear_model import LogisticRegression

def task_nine():
    vect = CountVectorizer(min_df=5, ngram_range=(1,3))

    X_train_vectorized = vect.fit_transform(X_train)
    X_train_transformed_add = add_feature(X_train_vectorized, [X_train.str.len(),X_train.str.isdigit()])

    X_test_vectorized = vect.transform(X_test)
    X_test_transformed_add = add_feature(X_test_vectorized, [X_test.str.len(), X_test.str.isdigit()])

    model = LogisticRegression(C=100)

    model.fit(X_train_transformed_add, y_train)

    y_predicted = model.predict(X_test_transformed_add)
    
    return roc_auc_score(y_test, y_predicted)

In [23]:
task_nine()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.972528988336757

### Task 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [24]:
def task_ten():
    
    spam = spam_data[spam_data['target'] != 0]
    not_spam = spam_data[spam_data['target'] == 0]
    
    word_spam = 0
    word_not= 0
    
    spam_data['length'] =spam_data['text'].str.findall(r'(\W)').str.len()
    
    
    return (np.mean(spam_data['length'][spam_data['target'] == 0]), np.mean(spam_data['length'][spam_data['target']==1]))

In [25]:
task_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [26]:
def task_eleven():
    vect = CountVectorizer(min_df=5, analyzer='char_wb',ngram_range=(2,5))

    X_train_vectorized = vect.fit_transform(X_train)
    X_train_transformed_add = add_feature(X_train_vectorized, [X_train.str.len(),
                                                               X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))
                                                               ,X_train.str.findall(r'(\W)').str.len()])

    X_test_vectorized = vect.transform(X_test)
    X_test_transformed_add = add_feature(X_test_vectorized, [X_test.str.len(), 
                                                             X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))
                                                            ,X_test.str.findall(r'(\W)').str.len()])
                                                        

    model = LogisticRegression(C=100)

    model.fit(X_train_transformed_add, y_train)

    y_predicted = model.predict(X_test_transformed_add)
    
    
    
    feature_names = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = model.coef_[0].argsort()
    smallest = feature_names[sorted_coef_index[:10]]
    largest = feature_names[sorted_coef_index[:-11:-1]]

    return (roc_auc_score(y_test, y_predicted), list(smallest), list(largest))

In [27]:
task_eleven()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(0.9813973821367333,
 ['..', '. ', ' i', ' go', '? ', ' y', 'pe', 'go', 'ca', ':)'],
 ['digit_count', 'co', 'ne', 'ww', 'ar', 'ia', ' ch', 'mob', 'xt', 'eply '])